---
title: "CS 546: Homework 1"
subtitle: "Metabolic Network Analysis"
author: "Trent VanHawkins"
date: today
format: 
    pdf:
        geometry: "margin=0.75in"
        mathspec: true
        code-overflow: wrap
        cap-location: bottom
        tbl-cap-location: bottom
        header-includes: 
        - \usepackage{fancyhdr, amsthm, amssymb,amsfonts,amsthm, amsmath, bbm}
        - \usepackage{float, tabularx}
        - \floatplacement{table}{H}
        - \pagestyle{fancy}
        - \fancyhead[R]{Homework 1}
        - \fancyhead[L]{Trent VanHawkins}
        - \fancyfoot[C]{\thepage} # Center page number at bottom of each page
page-layout: full
execute: 
  eval: true
---

# CSX46 Homework 1

In this homework assignment, you will be analyzing the human metabolic network. A simplified version of that network is provided for you in edge-list format in a two-column, tab-delimited text file `hsmetnet.txt` that is available at the following URL:  
[https://csx46.s3-us-west-2.amazonaws.com/hsmetnet.txt](https://csx46.s3-us-west-2.amazonaws.com/hsmetnet.txt)
In case you want to check that you have a complete and uncorrupted version of the file, here is it's MD5 checksum: `50bc7295c1f727cdc5867e4853a27583`. An example of the `hsmetnet.txt` file format is shown here:
```
alkylated DNA   REACTION1
REACTION1       DNAn
REACTION1       alkylated nucleobase
acetyl-CoA      REACTION2
1-alkyl-2-lyso-sn-glycero-3-phosphocholine      REACTION2
REACTION2       1-alkyl-2-acetyl-sn-glycero-3-phosphocholine
REACTION2       coenzyme A
deoxyribonucleoside triphosphate        REACTION3
(deoxynucleotides)(n)   REACTION3
```
You will see that there are two types of nodes; nodes that start with `REACTION` and nodes that do not. The former represent chemical *reactions*, and the latter represent *metabolites*. This graph is directed, so the ordering of the nodes is important; a row of the form
```
some-metabolite    REACTION523
```
is saying that metabolite `some-metabolite` is an *input* to (i.e., a reactant for) reaction `REACTION523`. Conversely, a row of the form
```
REACTION634    another-metabolite
```
is saying that metabolite `another-metabolite` is an *output* of (i.e., a product of) reaction `REACTION634`. A graph with two classes of nodes (and for which the only allowed edges are *between* nodes of the two classes, never *among* nodes of a single class) is called *bipartite*. So in this homework assignment we will be analyzing the human metabolic network as a bipartite graph.

You will need to submit your homework assignment as either a Jupyter notebook (preferred) or a PDF of a report showing both code *and* results from running the code.

For this homework assignment, you will need cairo, igraph, pandas, matplotlib, and numpy

In [ ]:
#!brew install cairo jpeg giflib
#!pip install pycairo
import cairo
#!pip install python-igraph
import igraph, pandas, matplotlib, numpy

Next, you will want to download the metabolic network from the CSX46 S3 bucket into your Google Colab instance:

In [ ]:
hsmetnet = pandas.read_csv(filepath_or_buffer= "https://csx46.s3-us-west-2.amazonaws.com/hsmetnet.txt", sep='\t', header=None)

hsmetnet.head()

Next, read in the metabolic network as an edge-list (hint: use `read_csv` from `pandas`) into a `pandas.DataFrame`. Name the two columns `source` and `target`. Show the first six rows of the data frame. Note, there are some duplicate rows in the data file (for whatever reason) so you will want to use the `drop_duplicates` method on the Pandas data frame.

Next, you will want to construct an igraph Graph from the pandas dataframe (hint: use `Graph.TupleList` with `directed=True`; you can use  `.values.tolist()` on the Pandas dataframe

**<font color='orange'>Question:** *How many distinct metabolites are there in the graph?* *How many reactions?* *How many edges are there?*</font>

(hint: use list comprehension, `in`, `for`, `len`, `str`, `set`, and `shape`)

(You can assume that no metabolite name has "REACTION" as a substring).

<font color="orange">**Question**: *In this graph, what are the top six metabolites in terms of vertex degree? (indegree+outdegree counted together)*</font>

(hint: use list comprehension, `for`, `enumerate`, `sorted` (with `reverse=True`), and `lambda` or `itemgetter`)

<font color="orange">**Task:** *Plot the distribution of the degrees of all of the metabolite vertices, on log-log scale*  (indegree+outdegree counted together).</font>

(hint: use `degree_distribution` in igraph, with `vertices=metabolite_inds`; use the `.bins()` method to get the bin counts out; you can use `matplotlib.pyplot.loglog` to plot)

(hint: do not try to plot bin counts of zero, as they cannot be plotted on a log scale; those zero counts should be removed before plotting)

<font color="orange">**Question**: *what is the exponent α of the best-fit power-law to the degree distribution?*</font>

(hint: use `igraph.statistics.power_law_fit`, and on the object that is returned, get the value of the `.alpha` variable)

<font color="orange">**Conceptual questions**: *How does the α that you get compare to the estimate of the power-law exponent reported by Jeong et al. in their 2000 article in Nature, “The large-scale organization of metabolic networks” (vol. 407, pp. 651–654) (see page 14 of reading-for-class-06.pdf)? Based on structure of the network that you analyzed (bipartite, containing reactions) vs. the structure of the network that they analyzed (network projected to a network containing only metabolites), is it appropriate to compare the exponents? Why or why not?*</font>

(You may also find page 12 of the supplementary material for the Jeong et al. article useful; that can be found [here](https://drive.google.com/file/d/1mEY3ikVKwUMVBiQDcM7-G-n1FFphm8LK/view?usp=share_link)).

<font color="#9a9a9a">[*Add your answers to this text cell*]</font>



**Task**: calculate the shortest-path-lengths between all pairs of metabolites (vertices) in the giant weakly connected component of the graph, discarding direction information in the graph.

(hint: use `igraph.Graph.connected_components` with `mode=weak` to get the giant weakly-connected component; then use `components.membership` to get a list (of length equal to the number of vertices) specifying which component each vertex is a member of. Then call `sizes()` on the object returned from `connected_components()` to get the sizes of each component, and use `numpy.argmax` to confirm that component 0 is the giant component. Then use `enumerate` to get the giant component members; then use `igraph.Graph.distances` with `mode=igraph.ALL`, with
`source=<metabolite_vertex_indices>` and `target=<metabolite_vertex_indices>` to get the
all-pairs-shortest-paths.)

<font color="orange">**Question:** *What is the average of the shortest-path-lengths between all pairs of metabolites in the giant (weakly connected) component of the network?*</font>

(Note: in calculating your average, do not include pairs of vertices that are the same vertex as both source and destination, which would have a distance of zero)

(hint: Use `numpy.array`, `numpy.isfinite` and `numpy.mean` to get the mean distance; throw away any shortest-paths distance value if it is infinite.)

<font color="orange">**Question**: *What is the maximum of the shortest-path-length between all pairs of metabolites (throwing away infinite values, as before) in the giant (weakly connected) component of the network?*</font>

(hint: use `numpy.max` and `numpy.isfinite`; Note, you are calculating the diameter of the giant component)

<font color="orange">**Conceptual question:**  *Why are the average geodesic distances that we get, roughly twice those reported in Fig. 3b of Jeong et al., 2000?*</font>

<font color="#9a9a9a">[*Add your answer to this text cell*]</font>


**Task:** Calculate the shortest-paths betweenness centrality for all metabolites in the directed network.  (hint: use the `betweenness` function with the `vertices=<vector of vertex indices>` option, with `directed=True`)

<font color="orange">**Task:** *plot the scatter plot of betweenness centrality vs. vertex degree for all metabolites, on log-log scale.*</font>  (In the plot, normalize your betweenness centrality values by dividing by M^2 where M is the number of metabolites.)

(hint:  use `numpy.array`, `numpy.where`, and `matplotlib.pyplot.scatter`)

<font color="orange">**Question**: *Among metabolites with degree k=2 , what metabolite has highest betweenness centrality in the network?*</font>

(hint:  use `numpy.argmax` and `numpy.where`)

<font color="orange">**Conceptual questions:**</font> Search on this metabolite in the HumanCyc database at [humancyc.org](https://humancyc.org), using the "Quick Search" box. Click on the hyperlinked metabolite that is displayed on the search results page. Click on the "reactions" tab, in the tabbed window in the lower part of the page. <font color="orange">What important metabolic cycles is this metabolite involved in?</font> Click on the "urea cycle". <font color="orange">What is the known consequence of absence of an enzyme in this pathway?</font>

<font color="#9a9a9a">[*Add your answers to this text cell*]</font>